# Undercomplete Autoencoder

L = MSE

In [ ]:
import numpy as np
import torch
import torch.utils.data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from torchvision.utils import make_grid , save_image
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
architecture_name = "simple_net"
ngpu=1
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
device

In [ ]:
workers = 8
batch_size = 32    
train_dataset = datasets.ImageFolder(root="/home/ninja/MNIST/training/",
                                     transform=transforms.Compose([transforms.ToTensor()]))
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=workers)

test_dataset = datasets.ImageFolder(root="/home/ninja/MNIST/testing/",
                                    transform=transforms.Compose([transforms.ToTensor()]))
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=workers)

In [ ]:
class Net(nn.Module):
    def __init__(self,input_dim=784,output_dim=784,ngpu=ngpu):
        super(Net, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.hidden1 = nn.Linear(self.input_dim, 100)
        self.act1 = nn.ReLU()
        self.hidden2 = nn.Linear(100, 100)
        self.act2 = nn.ReLU()
        self.out1 = nn.Linear(100, self.output_dim)

    def forward(self, x):
        h = self.act1(self.hidden1(x))
        h = self.act2(self.hidden2(h))
        out = torch.sigmoid(self.out1(h))
        return out

def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_normal_(m.weight).to(device)
        m.bias.data.fill_(0.001)
    
def define_model():
    # Instanciamos as redes
    net = Net().to(device)
    if (device.type == 'cuda') and (ngpu > 1):
        net = nn.DataParallel(net, list(range(ngpu)))

    # Inicializamos os pesos
    net.apply(init_weights)

    learning_rate, beta1, beta2 = [1e-3,0.5,0.999]
    eps,weight_decay,amsgrad = [1e-8,0,False]
    net_optimizer = optim.Adam(net.parameters(),
                               lr=learning_rate,
                               betas=(beta1,beta2),
                               eps=eps,
                               weight_decay=weight_decay,
                               amsgrad=amsgrad)
    return net,net_optimizer

mse_loss = nn.MSELoss()

In [ ]:
net,net_optimizer = define_model()

In [ ]:
for epoch in range(10):
    loss_ = []
    for _, (X_train,_) in enumerate(train_loader):
        X_train = X_train.view(-1,784).to(device)
        X_pred_train = net(X_train)
        loss = mse_loss(X_pred_train,X_train)
        loss.backward()
        net_optimizer.step()
        net_optimizer.zero_grad()

    print("Training loss for {} epoch: {}".format(epoch, loss))

In [ ]:
def show_adn_save(file_name,img):
    npimg = np.transpose(img.numpy(),(1,2,0))
    plt.imshow(npimg)

In [ ]:
show_adn_save("real",make_grid(X_train[:32].cpu().view(32,1,28,28).data))

In [ ]:
show_adn_save("generate",make_grid(net(X_train[:32]).cpu().view(32,1,28,28).data))

### Construir Encoder com transfer learning

In [ ]:
class Encoder(nn.Module):
    def __init__(self,input_dim=784,output_dim=100,ngpu=ngpu):
        super(Encoder, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.hidden1 = nn.Linear(self.input_dim, 100)
        self.act1 = nn.ReLU()
        self.hidden2 = nn.Linear(100, 100)

    def forward(self, x):
        h = self.act1(self.hidden1(x))
        h = self.hidden2(h)
        return h

In [ ]:
encoder = Encoder().to(device)

In [ ]:
def transfer_learning(trained_model,new_model):
    pretrained_dict = trained_model.state_dict()
    model_dict = new_model.state_dict()
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
    model_dict.update(pretrained_dict)
    new_model.load_state_dict(model_dict)
    return new_model

In [ ]:
encoder.state_dict()['hidden1.weight']

In [ ]:
encoder = transfer_learning(trained_model=net,new_model=encoder)

In [ ]:
net.state_dict()['hidden1.weight']

In [ ]:
encoder.state_dict()['hidden1.weight']

In [ ]:
for _, (X_train,y_train) in enumerate(train_loader):
    X_train = X_train.view(-1,784).to(device)
    break

In [ ]:
idx = 0
encoder(X_train[idx]).detach(),y_train[idx]

In [ ]:
idx = 1
encoder(X_train[idx]).detach(),y_train[idx]

In [ ]:
idx = -1
encoder(X_train[idx]).detach(),y_train[idx]